In [31]:
import pandas as pd
import numpy as np

from price_loaders.tradingview import load_asset_price
from meteostat import Point
from meteostat import Stations
from datetime import date

In [ ]:
def contractCloseData(contract):
    df = load_asset_price(contract, 10000, 'D')[['time', 'close']]
    df.time = df.time.apply(lambda x: date(x.year, x.month, x.day))
    return df

def marketSpreadYearFlag(longMonth, shortMonth, longExpYear, shortExpYear):
    expMonthToNum = {
        'U':  9,    # September
        'X':  11,   # November
        'F':  1,    # January
        'H':  3,    # March
        'K':  5,    # May
        'N':  7,    # Jul
        'Q':  8     # August
    }

    minYear = min(longExpYear, shortExpYear)
    maxYear = max(longExpYear, shortExpYear)

    maxMonth = max(expMonthToNum[longMonth], expMonthToNum[shortMonth])

    if (minYear == maxYear) and (maxMonth <= expMonthToNum['Q']): return f"{minYear-1}/{str(maxYear)[-2:]}"
    else:
        #print('erro') 
        if (minYear == longExpYear) and (expMonthToNum[longMonth] >= expMonthToNum['U']) and (expMonthToNum[shortMonth] <= expMonthToNum['Q']):
            return f"{maxYear-1}/{str(maxYear)[-2:]}"
        elif (minYear == shortExpYear) and (expMonthToNum[shortMonth] >= expMonthToNum['U']) and (expMonthToNum[longMonth] <= expMonthToNum['Q']):
            return f"{maxYear-1}/{str(maxYear)[-2:]}"
        else:
            return False

def calendarSpreadData(asset, longMonth, longExpYear, shortMonth, shortExpYear):
    if not (marketSpreadYearFlag(longMonth, shortMonth, longExpYear, shortExpYear)): "Please, input two contracts in the same market year"
    else: marketYear = marketSpreadYearFlag(longMonth, shortMonth, longExpYear, shortExpYear)

    long = contractCloseData(f"{asset}{longMonth}{longExpYear}")
    short = contractCloseData(f"{asset}{shortMonth}{shortExpYear}")

    DF = long.merge(short, on = 'time', suffixes = ('_long', '_short'))
    DF['spread'] = DF['close_long'] - DF['close_short']

    DF['marketYear'] = marketYear

    return DF

calendarSpreadData('ZS', 'N', 2026, 'Q', 2026)

,time,close_long,close_short,spread,marketYear
0,2023-11-14,1258.50,1248.75,9.75,2025/26
1,2023-11-15,1259.50,1249.75,9.75,2025/26
2,2023-11-16,1244.00,1234.25,9.75,2025/26
3,2023-11-17,1237.00,1227.25,9.75,2025/26
4,2023-11-20,1252.00,1242.25,9.75,2025/26
...,...,...,...,...,...
465,2025-09-23,1071.25,1069.50,1.75,2025/26
466,2025-09-24,1069.00,1067.25,1.75,2025/26
467,2025-09-25,1071.25,1069.25,2.00,2025/26
468,2025-09-26,1073.25,1071.25,2.00,2025/26


In [45]:
def marketSpreadYearFlag_Final(longMonth, shortMonth, longExpYear, shortExpYear):
    # Mapping the months to their position in the market year cycle (Sep=9 to Aug=8)
    expMonthToCalNum = {
        'U': 9,  # September (Start of Market Year)
        'X': 11, # November
        'F': 1,  # January (Next Calendar Year)
        'H': 3,  # March
        'K': 5,  # May
        'N': 7,  # July
        'Q': 8   # August (End of Market Year)
    }
    
    # 1. Determine the absolute later expiration date. 
    # To compare properly, we must account for 'F' being in the next calendar year, etc.
    def get_absolute_date(month, year):
        cal_month = expMonthToCalNum[month]
        cal_year = year
        if cal_month <= 8: # Months Jan-Aug (F, H, K, N, Q) are in the NEXT calendar year relative to the new crop year label
            cal_year += 1
        return (cal_year, cal_month)

    long_date = get_absolute_date(longMonth, longExpYear)
    short_date = get_absolute_date(shortMonth, shortExpYear)

    # 2. Identify the latest contract's month and its associated market year start
    if long_date >= short_date:
        latest_month_code = longMonth
        latest_year = longExpYear
    else:
        latest_month_code = shortMonth
        latest_year = shortExpYear

    # 3. Determine the Market Year Flag based on the month code of the latest contract:
    
    # If the contract is Sept (U) or Nov (X), the market year *starts* in that YYYY.
    if latest_month_code in ['U', 'X']:
        # Example: U26 (Sep 2026). Market Year: 2026/27. latest_year = 2026
        market_year_start = latest_year
        market_year_end = latest_year + 1
        
    # If the contract is Jan (F) through Aug (Q), it belongs to the market year that *started* in the prior YYYY.
    else: # 'F', 'H', 'K', 'N', 'Q'
        # Example: N26 (Jul 2026). Market Year: 2025/26. latest_year = 2026
        market_year_start = latest_year - 1
        market_year_end = latest_year
        
    return f"{market_year_start}/{str(market_year_end)[-2:]}"

marketSpreadYearFlag('N', 'X', 2025, 2025)

'2024/25'